# Inforet 2022: Project

## Imports

In [ ]:
import re, regex, timeit, gzip, random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher
from collections import Counter
from nltk.tokenize import MWETokenizer
from nltk.util import Trie
tqdm.pandas()
spacy.__version__ 

## Read and preprocess data

In [ ]:
# if you've already unzipped the file
patent_data=open('G06K.txt').read().strip()

# split into patents texts | 1 entry = 1 patent
patent_texts = patent_data.split('\n\n')

# split each patent into lines
patent_lines = patent_data.split('\n')

In [ ]:
print(len(patent_lines),'patent lines')
print(len(patent_texts),'texts of patents')

## Extract features

In [ ]:
cvectorizer = CountVectorizer(ngram_range=(2, 3), min_df=10, stop_words="english")
X=cvectorizer.fit_transform(patent_lines)

Xdf = pd.DataFrame(np.sum(X, axis=0), columns=cvectorizer.get_feature_names()).T.sort_values(by = 0, ascending = False)
Xdf.head(25)

### Manyterms

In [ ]:
# here are the potential terms
mwes = open('manyterms.lower.txt').read().lower().strip().split('\n')
print(mwes[44444:44456])
print(len(mwes),'mwes')

In [ ]:

# Here lowercase=False option is used to keep the original case of the terms, since we possibly could have term abbreviations. Like API, CAT, etc.
cvectorizer = CountVectorizer(ngram_range=(1, 4), stop_words="english", vocabulary=mwes, lowercase=True)
X=cvectorizer.fit_transform(patent_texts)

# Show top-25 most frequent terms
termdf_cv = pd.DataFrame(np.sum(X, axis=0), columns=cvectorizer.get_feature_names()).T.sort_values(by = 0, ascending = False)
termdf_cv.head(25)

- [EXPERIMENT] Longer words - more specific terms?

In [ ]:
# Count vectorizer with vocabulary
# Here lowercase=False option is used to keep the original case of the terms, since we possibly could have term abbreviations. Like API, CAT, etc.
cvectorizer = CountVectorizer(ngram_range=(3, 4), stop_words="english", vocabulary=mwes, lowercase=False)
X=cvectorizer.fit_transform(patent_lines)

# Show top-25 most frequent terms
term_cv_long = pd.DataFrame(np.sum(X, axis=0), columns=cvectorizer.get_feature_names()).T.sort_values(by = 0, ascending = False)
term_cv_long.head(25)

## 🪄 SpaCy NER

Instead of using EntityRuler, we can use the built-in PharaseMatcher and Span for annotation and saving it to the binary `.spacy` format

Let's start from understanding. Here is an example of showing part of text on one patent with default NER 

In [ ]:
from spacy.util import filter_spans
from spacy import displacy
from spacy.tokens import DocBin
from spacy.tokens import Span


nlp = spacy.load("en_core_web_lg")
doc = nlp(patent_texts[0][18000:20000]) # 
displacy.render(doc, style="ent", jupyter = True)

### Create DataSet

We need to create propper dataset that is compatible with SpaCy 3.0

In [ ]:
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(text) for text in termdf_cv.index]
matcher.add("Tech", patterns)

In [ ]:
# split train and test patent_lines with sciki-learn
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(patent_lines, test_size=0.3, random_state=42)


We are using PharsesMatcher to find entities similar to one from mayterms.txt  
Then Span is labeled and saved into the binary `.spacy` format

Training data

In [ ]:
LABEL = "TECH"
doc_bin_train = DocBin() # create a DocBin object

# nlp.max_length = 2000000
for training_example  in tqdm(train_lines[:30000]): #~50 patents
    doc = nlp.make_doc(training_example) 
    ents = []
    
    for match_id, start, end in matcher(doc):
        #print(i,"Matched based on lowercase token text:", doc[:10], '::::::::',doc[start:end],start, end)
        span = Span(doc, start, end, label=LABEL)
        #print(span, span.label_)
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)

    filtered_ents = filter_spans(ents)
    print(filtered_ents[:3])
    doc.ents = filtered_ents 
    doc_bin_train.add(doc)

Validation

In [ ]:
LABEL = "TECH"
doc_bin_valid = DocBin() # create a DocBin object
nlp = spacy.blank("en")
# nlp.max_length = 2000000
for training_example  in tqdm(test_lines[:10000]): #~15 patents
    doc = nlp.make_doc(training_example) 
    ents = []
    
    for match_id, start, end in matcher(doc):
        #print(i,"Matched based on lowercase token text:", doc[:10], '::::::::',doc[start:end],start, end)
        span = Span(doc, start, end, label=LABEL)
        #print(span, span.label_)
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)

    filtered_ents = filter_spans(ents)
    print(filtered_ents[:3])
    doc.ents = filtered_ents 
    doc_bin_valid.add(doc)

Test

In [ ]:
LABEL = "TECH"
doc_bin_test = DocBin() # create a DocBin object
nlp = spacy.blank("en")
# nlp.max_length = 2000000
for training_example  in tqdm(test_lines[10000:12000]): #~5 patents
    doc = nlp.make_doc(training_example) 
    ents = []
    
    for match_id, start, end in matcher(doc):
        #print(i,"Matched based on lowercase token text:", doc[:10], '::::::::',doc[start:end],start, end)
        span = Span(doc, start, end, label=LABEL)
        #print(span, span.label_)
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)

    filtered_ents = filter_spans(ents)
    print(filtered_ents[:3])
    doc.ents = filtered_ents 
    doc_bin_test.add(doc)

Save data

In [ ]:
doc_bin_train.to_disk("training_data.spacy") # save the docbin object
doc_bin_valid.to_disk("valid_data.spacy") # save the docbin object
doc_bin_test.to_disk("test_data.spacy") # save the docbin object

# save train_lines to txt file
with open('train_lines.txt', 'w') as f:
    for line in train_lines:
        f.write(line)
        f.write('\n')
f.close()

# save train_lines to txt file
with open('valid_lines.txt', 'w') as f:
    for line in test_lines[:10000]:
        f.write(line)
        f.write('\n')
f.close()

# save test_lines to txt file
with open('test_lines.txt', 'w') as f:
    for line in test_lines[10000:12000]:
        f.write(line)
        f.write('\n')
f.close()

### Configuration

Donwnload __base_config.cfg__ for your system at https://spacy.io/usage/training#quickstart

In [ ]:
# Run to generate full training config
!python -m spacy init fill-config base_config.cfg config.cfg

### Training

Run training. All results are stored into __./spacy_output__ 

In [ ]:
!python -m spacy train config.cfg --output ./spacy_output --paths.train ./training_data.spacy --paths.dev ./valid_data.spacy

### Testing

In [ ]:
nlp_ner = spacy.load("./spacy_output/model-best")

colors = {"TECH": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors} 

for line in test_lines[10000:10005]:
    doc = nlp_ner(line)
    spacy.displacy.render(doc, style="ent", options= options, jupyter=True)


## 🦄 Prodigy: Make it even better

For this part i have used this tutorial: https://newscatcherapi.com/blog/train-custom-named-entity-recognition-ner-model-with-spacy-v3  
and official documentation: https://spacy.io/usage/training#custom-ner-model

### 📖 Teach it! 

One of the besst feature of Prodigy is that you can focus annotation on the most uncertain enitties.  
For this one we use __ner.teach__

As a dataset, used valid_data.txt, since model is already fitter on the training data

In [ ]:
!prodigy ner.teach ner_tech  ./spacy_output/model-best  valid_lines.txt --label TECH

<img src="./img/binary.png" height=400>

Also, instead of binary judging, we can correct model prediction manualy by using __ner.correct__

In [ ]:
!prodigy ner.correct gold_tech  ./spacy_output/model-best  valid_lines.txt --label TECH

<img src="./img/annotation.png" height=420>

### 🤝 Merge it!

Now, we need to merge our binary annotation into the __gold dataset__  
This means that now we are fixing annotation manually in the text which we are rejected during __ner.teach__  

Those annotation can be directly merged into already created dataset(by ner.correct)

In [ ]:
!prodigy ner.silver-to-gold gold_tech ner_tech ./spacy_output/model-best --label TECH 

### 🏋️‍♀️ .. or train it with Prodigy

We can finetune\ train our existing SpaCy model(pipeline) inside prodigy  

Here we train existing `/model_best` and output our finte-tuned model into `spacy_output`

In [ ]:
!prodigy train ./prodigy_output --ner gold_tech --eval-split 0.3 --base-model ./spacy_output/model-best 

<img src="./img/terminal_training.png" height=400>


### ✍️ Evaluation 
- Let's evaluate on the test-lines. They are already randomized, so it's a good place to start.

In [ ]:
nlp_ner = spacy.load("./prodigy_output/model-best")

# load test lines
with open('test_lines.txt', 'r') as f:
    test_lines = f.readlines()
f.close()


colors = {"TECH": "#7DF6D9", "MEDICALCONDITION":"#FFFFFF"}
options = {"colors": colors} 

for line in test_lines[10000:10010]:
    doc = nlp_ner(line)
    spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

## 🏆 Evaluation

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')

# load test lines
with open('test_lines.txt', 'r') as f:
    test_lines = f.readlines()
f.close()

In [ ]:
cd_rom = wn.synsets('CD-ROM', pos='n')
computer = wn.synsets('computer', pos='n')

In [ ]:
for synset in cd_rom:
    for synset2 in computer:
        print(synset, synset2)
        print("Score:", synset.wup_similarity(synset2))

In [ ]:
wn.synsets('new_york', pos='n')

#### Spacy embeddings

In [ ]:
# load best model

nlp_ner = spacy.load("./spacy_output/model-best")

In [ ]:
for line in test_lines[:3]:
    doc = nlp_ner(line)
    spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [ ]:
line

In [ ]:
entities_list =  nlp_ner(line).ents

# for each 2 entities in entities_list claculate similarity

for i in range(0, len(entities_list)-1):
    for j in range(i+1, len(entities_list)):
        print(entities_list[i], "<->", entities_list[j], " : ", entities_list[i].similarity(entities_list[j]))

In [ ]:
# load large model for comparison
nlp = spacy.load("en_core_web_lg")

word_1 = nlp("cloud platform")
word_2 = nlp("service provider")

print(word_1, "<->", word_2, word_1.similarity(word_2))